## Tabelas de Streaming

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/declarative-pipelines/declarative-pipelines-1.png?raw=true" width="400px" style="width:400px; float: right;" />

Uma tabela de streaming é uma tabela Delta com suporte adicional para processamento de dados em streaming ou incremental. Uma tabela de streaming pode ser alvo de um ou mais fluxos em um pipeline ETL.

Tabelas de streaming são uma boa escolha para ingestão de dados pelos seguintes motivos:
* Cada linha de entrada é processada apenas uma vez, o que modela a grande maioria das cargas de trabalho de ingestão (ou seja, adicionando ou fazendo upsert de linhas em uma tabela).
* Elas podem lidar com grandes volumes de dados apenas de acréscimo.

Tabelas de streaming também são uma boa escolha para transformações de streaming de baixa latência pelos seguintes motivos:
* Permitem raciocinar sobre linhas e janelas de tempo
* Lidam com grandes volumes de dados
* Baixa latência

<!-- Coletar dados de uso (visualização). Remova para desabilitar a coleta. Veja o README para mais detalhes.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-engineering&org_id=1444828305810485&notebook=%2Ftransformations%2F00-pipeline-tutorial&demo_name=pipeline-bike&event=VIEW&path=%2F_dbdemos%2Fdata-engineering%2Fpipeline-bike%2Ftransformations%2F00-pipeline-tutorial&version=1&user_hash=23c97c22545194e8a2febe533c183aca28a287ded4335bcc65c8b60cdcfa14c7">

## Criando nossas tabelas de streaming bronze
Veja [bronze.sql]($./transformations/01-bronze.sql) para entender como criamos nossas tabelas bronze `maintenance_logs_raw`, `rides_raw`, `weather_raw` e `customers_cdc_raw`.

# Enriquecendo nossos dados com funções de IA

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/declarative-pipelines/declarative-pipelines-2.png?raw=true" width="400px" style="width:400px; float: right;" />

Agora que carregamos nossos dados brutos, vamos enriquecê-los em nossa camada silver.

Nossos registros de manutenção incluem um campo não estruturado `issue_description`, que não é muito útil para análises. Vamos usar a função `ai_classify` para categorizar cada um desses problemas em categorias para relatórios.

In [0]:
%run ./00_variaveis

In [0]:
spark.sql(f"""
select
  date(reported_time) as maintenance_date,
  maintenance_id,
  bike_id,
  reported_time,
  resolved_time,
  issue_description,
  -- Classifique os problemas como relacionados a freios, correntes/pedais, pneus ou outro
  ai_classify(issue_description, array("brakes", "chains_pedals", "tires", "other")) as issue_type
from
  {catalog_name}.{schema_name}.maintenance_logs_raw
limit 10
""")

## Criando nossas tabelas de streaming silver enriquecidas com IA
Veja [silver.sql]($./transformations/02-silver.sql) para ver como criamos nossas tabelas silver `maintenance_logs`, `rides`, `weather` e `customers` (SCD Tipo 2 usando Auto CDC).

# Processando agregações incrementalmente com Visualizações Materializadas

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/declarative-pipelines/declarative-pipelines-3.png?raw=true" width="400px" style="width:400px; float: right;" />

Assim como as visualizações padrão, as visualizações materializadas são o resultado de uma consulta e você as acessa da mesma forma que acessaria uma tabela. Diferente das visualizações padrão, que recalculam os resultados a cada consulta, as visualizações materializadas armazenam os resultados em cache e os atualizam em um intervalo especificado. Como uma visualização materializada é pré-computada, consultas sobre ela podem ser executadas muito mais rapidamente do que em visualizações regulares.

Uma visualização materializada é um objeto de pipeline declarativo. Ela inclui uma consulta que a define, um fluxo para atualizá-la e os resultados em cache para acesso rápido. Uma visualização materializada:
* Acompanha mudanças nos dados de origem.
* Ao ser acionada, processa incrementalmente os dados alterados e aplica as transformações necessárias.
* Mantém a tabela de saída sincronizada com os dados de origem, com base em um intervalo de atualização especificado.

Visualizações materializadas são uma boa escolha para muitas transformações:
* Você aplica raciocínio sobre resultados em cache ao invés de linhas. Na verdade, você simplesmente escreve uma consulta.
* Elas estão sempre corretas. Todos os dados necessários são processados, mesmo que cheguem atrasados ou fora de ordem.
* Frequentemente são incrementais. O Databricks tentará escolher a estratégia apropriada que minimize o custo de atualização de uma visualização materializada.

Quando o pipeline que define uma visualização materializada é acionado, a visualização é automaticamente mantida atualizada, geralmente de forma incremental. O Databricks tenta processar apenas os dados necessários para manter a visualização materializada atualizada. Uma visualização materializada sempre mostra o resultado correto, mesmo que seja necessário recomputar totalmente o resultado da consulta, mas frequentemente o Databricks faz apenas atualizações incrementais, o que pode ser muito menos custoso do que uma recomputação completa.

## Criando nossas Visualizações Materializadas gold
Veja [gold.sql]($./transformations/03-gold.sql) para ver como criamos nossas tabelas gold `bikes`, `stations` e `maintenance_events`

## Analisando seus indicadores de negócio
Você tem tudo o que precisa! Quando estiver pronto, abra seu <a  dbdemos-dashboard-id="bike-rental" href='/sql/dashboardsv3/01f0931f3a0516209179aedea0ed1b08' target="_blank">Painel de Negócios de Aluguel de Bicicletas</a> para acompanhar todos os seus insights em tempo real!
